# 1. Install and import require packages for EDA

In [92]:
# install require packages
!pip install pandas
!pip install country_converter
!pip install plotly
!pip install matplotlib
!pip install seaborn

You should consider upgrading via the 'C:\Users\ad\PycharmProjects\RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ad\PycharmProjects\RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ad\PycharmProjects\RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ad\PycharmProjects\RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


You should consider upgrading via the 'C:\Users\ad\PycharmProjects\RS\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [ ]:
# import require packages

import pandas as pd
import numpy as np
import country_converter as coco
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# 2. Load dataset as dataframes

In [ ]:
# data paths
book_hist_path = '../book_crossing/book_history.dat'
book_rate_path = '../book_crossing/book_ratings.dat'
items_info_path = '../book_crossing/items_info.dat'
users_info_path = '../book_crossing/users_info.dat'

In [ ]:
book_hist_df = pd.read_csv(book_hist_path, delimiter='\t', header=None, low_memory=False, na_values=np.nan)
print(book_hist_df.head())

In [ ]:
book_rate_df = pd.read_csv(book_rate_path, delimiter='\t', low_memory=False)
print(book_rate_df.head())

In [ ]:
user_info_df = pd.read_csv(users_info_path, delimiter='\t', index_col=None, na_values=np.nan)

In [ ]:
print(user_info_df.columns)

In [ ]:
user_info_df.head()

In [ ]:
items_info_df = pd.read_csv(items_info_path, delimiter='\t')

print(items_info_df.head())

In [ ]:
print(items_info_df.columns)

# 3. EDA

## 3.1  Exploring USER INFO

In [ ]:
# get number of users
print(user_info_df.shape)
print(user_info_df.columns)
print(user_info_df.head())

##### >> Total: 2946 users, each user has 3 attributes: 'User-ID', 'Location', 'Age'.

#### 3.1.1. Explore Age from users info

In [ ]:
# Print out age values
print(sorted(user_info_df.Age.unique()))

##### >> 'Age' columns no have any null values, and have some value very high, more than 100.

##### // As we see, people who are less than 5 years old or more than 90 years old usually can't rate books much, so in this case we will replace them with nan values.

In [ ]:
user_info_df.loc[(user_info_df.Age > 90) | (user_info_df.Age < 5), 'Age'] = np.nan

In [ ]:
print(user_info_df[(user_info_df.Age > 90) | (user_info_df.Age < 5)])

In [ ]:
print(user_info_df[user_info_df['Age'] == np.nan])

##### Replace nan values in Age column with mean()

In [ ]:
user_info_df['Age'] = user_info_df['Age'].fillna(user_info_df['Age'].mean())

In [ ]:
user_info_df.head()

In [ ]:
# convert type 'Age' to integer
user_info_df['Age'] = user_info_df['Age'].astype(int)

In [ ]:
user_info_df.head()

#### 3.1.2. Explore User's Location

In [ ]:
# explore countries from location
countries = []
user_info_df['Country'] = user_info_df['Location'].apply(lambda x: x.split(',')[-1])
# countries.append(add[-1] for add in addresses)

In [ ]:
print(user_info_df.head())

In [ ]:
# print out all location from all users
locations = sorted(user_info_df.Location.unique())
countries = sorted(user_info_df.Country.unique())

print(f'We have total: {len(locations)} locations')
print(f'We have total: {len(countries)} countries')

##### >> So, we have 1799 unique locations from 2946 users in dataset

In [ ]:
print(user_info_df['Country'].value_counts())

#### As we see, top 3 countries has most book reviewers: usa (2128), canada (284), united kingdom(113).

In [ ]:
print(user_info_df[user_info_df['Country']==''])

##### And has 72 people who we miss about their country name

In [ ]:
converted_country=coco.convert(names=user_info_df["Country"], to="ISO3")
user_info_df["Country"]=converted_country

In [ ]:
country=user_info_df["Country"].value_counts()
# user_info_df.loc[user_info_df['Country']=='not found'] = 'USA'

# country=user_info_df["Country"].value_counts()
user_info_df['Country'].loc[user_info_df['Country']=='not found'] = 'USA'


print(country)
print(type((country)))
fig=px.choropleth(locations=country.index,
                  color=country.values,
                  color_continuous_scale=px.colors.sequential.dense,
                  template='plotly_dark',
                  title='Distribution of users by countries')

fig.update_layout(font=dict(size=17, family="Franklin Gothic"))
fig.show()

In [ ]:
user_info_df
print(user_info_df['Country'].value_counts())

##  3.2  Explore ITEM INFO

In [ ]:
print(items_info_df.head())

In [ ]:
# describe item info df
items_info_df.describe()

In [ ]:
items_info_df.info()

In [ ]:
# check null values in item info
items_info_df.isnull().sum()

In [ ]:
# check null values in item info
items_info_df.isnull().sum()

In [ ]:
# # convert 'Year-Of-Publication' values to numeric, if cannnot convert, this value cell will be np.nan
# items_info_df['Year-Of-Publication'] = pd.to_numeric(df['Year-Of-Publication'], errors='coerce')

In [ ]:
items_info_df

In [ ]:
book_title_df = pd.DataFrame(items_info_df["Book-Title"].value_counts())

In [ ]:
# top 5 book's title
book_title_df.head()

In [ ]:
def plot_top_values(df, column, title, x_label, y_label, top=20, template = 'plotly_dark'):
    
    pl = df[column].value_counts().head(top)
    fig = px.bar(y=pl.values, 
                 x=pl.index, 
                 color_discrete_sequence=px.colors.sequential.PuBuGn,
                 text=pl.values,
                 title= title,
                 template= template)
    fig.update_layout(
        xaxis_title=x_label,
        yaxis_title=y_label,
        font = dict(size=15,family="Franklin Gothic"))
    fig.show()

In [ ]:
plot_top_values(items_info_df, 'Year-Of-Publication', 'Top 20 years in which books were published in terms of count', "Year", "Count")

In [ ]:
# as we see, number of books were published increasing every year

In [ ]:
print(items_info_df['Year-Of-Publication'].unique())

In [ ]:
print(items_info_df[items_info_df['Year-Of-Publication']=='Hutchinson'].index)
print(items_info_df[items_info_df['Year-Of-Publication']=='Baldini&amp Castoldi'].index)
print(items_info_df[items_info_df['Year-Of-Publication']=='Bantam Book'].index)
print(items_info_df[items_info_df['Year-Of-Publication']=='http://images.amazon.com/images/P/0451404874.01.THUMBZZZ.jpg'].index)

In [ ]:
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='Hutchinson'].index)
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='Baldini&amp Castoldi'].index)
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='http://images.amazon.com/images/P/0451404874.01.THUMBZZZ.jpg'].index)

In [ ]:
items_info_df["Book-Author"] = items_info_df["Book-Author"].apply(lambda x: x.strip())

In [ ]:
print(items_info_df["Book-Author"].unique())

In [ ]:
print(items_info_df[items_info_df["Publisher"]=='1stBooks Library'])

## 3.3 EXPLORE with RATING INFO

In [ ]:
book_rate_df.head()

In [ ]:
print(book_rate_df.describe())

In [ ]:
print(book_rate_df.info())

In [ ]:
print(book_rate_df['rating'].unique())

In [ ]:
print(book_rate_df['rating'].value_counts())

In [ ]:
user_group = book_rate_df.groupby("user")
item_group = book_rate_df.groupby("item")


average_user_rating = user_group["rating"].mean()
number_of_ratings_by_user = user_group["rating"].count()
average_book_rating = item_group["rating"].mean()
number_of_book_ratings = item_group["rating"].count()

average_user_rating.name = "avg_rating"
number_of_ratings_by_user.name = "N_ratings"
average_book_rating.name = "avg_rating"
number_of_book_ratings.name = "N_ratings"

In [ ]:
average_user_rating

In [ ]:
# Merge with orig df
user_info_df = user_info_df.rename(columns={'User-ID': 'user'})
print(user_info_df.head())
user_info_df = user_info_df.join(number_of_ratings_by_user, on="user")
# user_info_df = user_info_df.join(average_user_rating, on="user")
# books = books.join(number_of_book_ratings, on="ISBN")
# books = books.join(average_book_rating, on="ISBN")

# users["N_ratings"] = users["N_ratings"].fillna(0)
# books["N_ratings"] = books["N_ratings"].fillna(0)

# users["N_ratings"] = users["N_ratings"].astype("int64")
# books["N_ratings"] = books["N_ratings"].astype("int64")

In [ ]:
user_info_df = user_info_df.join(average_user_rating, on="user")


In [ ]:
books = books.join(number_of_book_ratings, on="ISBN")
books = books.join(average_book_rating, on="ISBN")


In [553]:
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='Hutchinson'].index)
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='Baldini&amp Castoldi'].index)
items_info_df = items_info_df.drop(items_info_df[items_info_df['Year-Of-Publication']=='http://images.amazon.com/images/P/0451404874.01.THUMBZZZ.jpg'].index)

In [554]:
items_info_df["Book-Author"] = items_info_df["Book-Author"].apply(lambda x: x.strip())

In [555]:
print(items_info_df["Book-Author"].unique())

["Carlo D'Este" 'E. J. W. Barber' 'Robert Cowley' ... 'F. Engels'
 'Essie Summers' 'Katherine Sherwood Bonner McDowell']


In [556]:
print(items_info_df[items_info_df["Publisher"]=='1stBooks Library'])

       Book_ID        ISBN                                         Book-Title  \
33          34  1414035004  The Adventures of Drew and Ellie: The Magical ...   
2716      2717  1410798860  The Diary of Alice Mary: A Factual Childhood D...   
4703      4704  140332557X  The Little Guide to Happiness: How to Smile Again   
10080    10081  1410744310   Lost in the Rain II: Poetry and Quotes to Ponder   
10290    10291  1588204030  $oft Money: The True Power in Our Nation's Cap...   
11239    11240  1410716856                              Starlight, Starbright   
11602    11603  141401354X                                  Ocean of the Dead   
13198    13199  1410785971             The Story in the Little Brown Suitcase   

                     Book-Author Year-Of-Publication         Publisher  \
33                Charles Noland                2003  1stBooks Library   
2716    Alice Mary Connally Fisk                2003  1stBooks Library   
4703       Michael Kevin Naselli                

## 3.3 EXPLORE with RATING INFO

In [557]:
book_rate_df.head()

,user,item,rating
0,1,6264,7.0
1,1,4350,7.0
2,1,6252,5.0
3,1,202,9.0
4,1,6266,6.0


In [558]:
print(book_rate_df.describe())

               user          item        rating
count  62656.000000  62656.000000  62656.000000
mean    1458.705056   6106.118839      7.953700
std      848.564252   4847.444402      1.717846
min        1.000000      1.000000      1.000000
25%      760.000000   1888.000000      7.000000
50%     1398.000000   4977.000000      8.000000
75%     2218.000000   9538.000000      9.000000
max     2945.000000  17384.000000     10.000000


In [559]:
print(book_rate_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62656 entries, 0 to 62655
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   user    62656 non-null  int64  
 1   item    62656 non-null  int64  
 2   rating  62656 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB
None


In [560]:
print(book_rate_df['rating'].unique())

[ 7.  5.  9.  6.  8. 10.  4.  3.  1.  2.]


In [561]:
print(book_rate_df['rating'].value_counts())

rating
8.0     15629
10.0    13884
9.0     12008
7.0      9906
5.0      5183
6.0      4236
4.0       859
3.0       552
2.0       239
1.0       160
Name: count, dtype: int64


In [562]:
user_group = book_rate_df.groupby("user")
item_group = book_rate_df.groupby("item")


average_user_rating = user_group["rating"].mean()
number_of_ratings_by_user = user_group["rating"].count()
average_book_rating = item_group["rating"].mean()
number_of_book_ratings = item_group["rating"].count()

average_user_rating.name = "avg_rating"
number_of_ratings_by_user.name = "N_ratings"
average_book_rating.name = "avg_rating"
number_of_book_ratings.name = "N_ratings"

In [563]:
average_user_rating

user
1        8.062500
2        9.282051
5        8.500000
7        7.045455
9        7.040000
          ...    
2937     8.662500
2939    10.000000
2942     7.250000
2943     7.956522
2945     7.166667
Name: avg_rating, Length: 1295, dtype: float64

In [564]:
# Merge with orig df
user_info_df = user_info_df.rename(columns={'User-ID': 'user'})
print(user_info_df.head())
user_info_df = user_info_df.join(number_of_ratings_by_user, on="user")
# user_info_df = user_info_df.join(average_user_rating, on="user")
# books = books.join(number_of_book_ratings, on="ISBN")
# books = books.join(average_book_rating, on="ISBN")

# users["N_ratings"] = users["N_ratings"].fillna(0)
# books["N_ratings"] = books["N_ratings"].fillna(0)

# users["N_ratings"] = users["N_ratings"].astype("int64")
# books["N_ratings"] = books["N_ratings"].astype("int64")

   user                     Location  Age Country
0     1  minneapolis, minnesota, usa   24     USA
1     2   san diego, california, usa   20     USA
2     3      novinger, missouri, usa   16     USA
3     4      sonoma, california, usa   34     USA
4     5    berkeley, california, usa   23     USA


In [565]:
user_info_df = user_info_df.join(average_user_rating, on="user")


In [ ]:
books = books.join(number_of_book_ratings, on="ISBN")
books = books.join(average_book_rating, on="ISBN")
